In [1]:
import sys
sys.path.append('../../..')
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_orderbooks_from_file, get_binance_orderbook, convert_config
from hydradx.model.processing import get_omnipool_data_from_file, get_centralized_market
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb
import hydradx.model.amm.arbitrage_agent_general as arb2

from pprint import pprint

import json
import ast

Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = False

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
# load Omnipool
print('loading omnipool...')
RPC="wss://rpc.hydradx.cloud"
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=False)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("archive/")
    
# pprint(tokens)
init_state = OmnipoolState(
    tokens=tokens,
    lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list},
    asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list},
    preferred_stablecoin='USDT',
)

loading omnipool...


In [5]:
# arb_file = "arbconfig.txt"
arb_file = "arbconfig3.txt"

with open('config/' + arb_file, 'r') as json_file:
    cfg = json.load(json_file)

for d in cfg:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])
    d['tkn_pair'] = (asset_map[d['tkn_ids'][0]], asset_map[d['tkn_ids'][1]])

cfg2 = convert_config(cfg)  # ast.literal_eval(ast_file.read())
print('configs loaded')


configs loaded


In [6]:
# load centralized markets
print('loading centralized markets...')
cex_fees = {
    'kraken': 0.0016,
    'binance': 0.0010
}

cex_dict = {}
for exchange in ('kraken', 'binance'):
    if get_live_markets:
        cex_dict[exchange] = get_centralized_market(
            config=cfg, 
            exchange_name=exchange, 
            trade_fee=cex_fees[exchange],
            archive=False
        )
    else:
        cex_dict[exchange] = CentralizedMarket(
            order_book=get_orderbooks_from_file("archive/")[exchange],
            unique_id=exchange,
            trade_fee=cex_fees[exchange]
        )

loading centralized markets...


In [7]:
print('calculating arbitrage...')
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state1 = init_state.copy()
all_swaps1 = arb.get_arb_swaps(op_state1, cex_dict1, cfg)

pprint(all_swaps1)
print(f"strategy1 = {len(all_swaps1)} swaps")

exchanges = {
    'omnipool': init_state.copy(),
    'binance': cex_dict['binance'].copy(),
    'kraken': cex_dict['kraken'].copy()
}
max_liquidity = {ex_name: {tkn: 100000000 for tkn in ex.asset_list} for ex_name, ex in exchanges.items()}
cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state2 = init_state.copy()
# ops = arb2.get_arb_opps(
#     exchanges={'omnipool': op_state2, **cex_dict2}, config=cfg2, 
#     max_liquidity=max_liquidity
# )
# print(ops)
all_swaps2 = arb2.get_arb_swaps(exchanges=exchanges, config=cfg2, max_liquidity=max_liquidity)
# print("strategy 2")
pprint(all_swaps2)
print(f"refactored strategy = {len(all_swaps2)} swaps")
# print(cfg)


calculating arbitrage...
[{'cex': {'amount': 39.539,
          'buy_asset': 'BNC',
          'price': 0.5792894999999999,
          'sell_asset': 'USD',
          'trade': 'buy'},
  'dex': {'amount': 39.539,
          'buy_asset': 'USDT',
          'min_buy': 26.169985439240936,
          'price': 0.579,
          'sell_asset': 'BNC',
          'trade': 'sell'},
  'exchange': 'kraken'},
 {'cex': {'amount': 55.314,
          'buy_asset': 'BNC',
          'price': 0.58029,
          'sell_asset': 'USD',
          'trade': 'buy'},
  'dex': {'amount': 55.314,
          'buy_asset': 'USDT',
          'min_buy': 36.60369851100445,
          'price': 0.58,
          'sell_asset': 'BNC',
          'trade': 'sell'},
  'exchange': 'kraken'},
 {'cex': {'amount': 5453.044,
          'buy_asset': 'BNC',
          'price': 0.5812904999999999,
          'sell_asset': 'USD',
          'trade': 'buy'},
  'dex': {'amount': 5453.044,
          'buy_asset': 'USDT',
          'min_buy': 3566.5935982196333,

In [8]:
state1 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list}
for exchange in cex_dict:
    for asset in cex_dict[exchange].asset_list:
        if asset not in holdings:
            holdings[asset] = 1000000000
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}

arb.execute_arb(state1, cex_dict1, agent1, all_swaps1)
agent2 = init_agent.copy()
exchanges2 = {ex_name: ex.copy() for ex_name, ex in exchanges.items()}
arb2.execute_arb(exchanges2, agent2, all_swaps2)

equivalency_map = {
    'WETH': 'ETH',
    'XETH': 'ETH',
    'XXBT': 'BTC',
    'WBTC': 'BTC',
    'ZUSD': 'USD',
    'USDT': 'USD',
    'USDC': 'USD',
    'DAI': 'USD',
    'USDT001': 'USD',
    'DAI001': 'USD',
    'WETH001': 'ETH',
    'WBTC001': 'BTC',
    'iBTC': 'BTC',
    'XBT': 'BTC',
}

profit1 = arb.calculate_profit(init_agent, agent1, equivalency_map)
profit2 = arb.calculate_profit(init_agent, agent2, equivalency_map)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Binance spot")
print(cex_dict1['binance'].value_assets(profit1, equivalency_map))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Binance spot")
print(exchanges['binance'].value_assets(profit2, equivalency_map))

# try rerouting to see if we can squeeze some more profit out of it
agent3 = init_agent.copy()
exchanges3 = {
    'omnipool': init_state.copy(),
    'binance': cex_dict['binance'].copy(),
    'kraken': cex_dict['kraken'].copy()
}
all_swaps3 = arb2.combine_swaps(
    exchanges=exchanges3,
    agent=agent3,
    swaps=all_swaps2,
    equivalency_map=equivalency_map
)
print(f"combination strategy = {len(all_swaps3)} swaps")
arb2.execute_arb(exchanges3, agent3, all_swaps3)
profit3 = arb2.calculate_profit(init_agent, agent3, equivalency_map)

print("strategy 3 profit")
print(profit3)
print("strategy 3 profit at Binance spot")
print(exchanges3['binance'].value_assets(profit3, equivalency_map))


strategy 1 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 559.3892337083817, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'PHA': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0.0}
strategy 1 profit at Binance spot
559.3892337083817
strategy 2 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 558.619425535202, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'PHA': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0.0}
strategy 2 profit at Binance spot
558.619425535202
combination strategy = 31 swaps
strategy 3 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 558.619425535202, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'PHA': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 37.131226658821106}
strategy 3 profit at Binance spot
564.6328295491594


In [9]:
# import time
# ts = time.time()
# with open(f'./archive/omnipool_data_tokens_{ts}.json', 'w') as output_file:
#     json.dump(tokens, output_file)
# with open(f'./archive/omnipool_data_fees_{ts}.json', 'w') as output_file:
#     json.dump(fees, output_file)
# with open(f'./archive/omnipool_data_assetmap_{ts}.json', 'w') as output_file:
#     json.dump(asset_map, output_file)

In [10]:
# kraken = cex_dict['kraken']
# for tkn_pair in kraken.order_book:
#     with open(f'./archive/kraken_orderbook_{tkn_pair[0]}-{tkn_pair[1]}_test.json', 'w') as output_file:
#         y = {'result': {f"{tkn_pair[0]}{tkn_pair[1]}": {'bids': kraken.order_book[tkn_pair].bids, 'asks': kraken.order_book[tkn_pair].asks}}}
#         json.dump(y, output_file)

In [11]:
# binance = cex_dict['binance']
# for tkn_pair in binance.order_book:
#     with open(f'./archive/binance_orderbook_{tkn_pair[0]}-{tkn_pair[1]}_test.json', 'w') as output_file:
#         y = {'bids': binance.order_book[tkn_pair].bids, 'asks': binance.order_book[tkn_pair].asks}
#         json.dump(y, output_file)

In [12]:
from hydradx.model.amm.arbitrage_agent import flatten_swaps
arb_swaps_4 = arb2.combine_swaps(
    exchanges=exchanges,
    agent=init_agent,
    swaps=[{**swap, 'exchange': (swap['exchange'] if swap['exchange'] != 'dex' else 'omnipool')} for swap in flatten_swaps(all_swaps1)],
    equivalency_map=equivalency_map
)
exchanges4 = {ex_name: ex.copy() for ex_name, ex in exchanges.items()}
agent4 = init_agent.copy()
arb2.execute_arb(exchanges4, agent4, arb_swaps_4)
profit4 = arb.calculate_profit(init_agent, agent4, equivalency_map)
print("strategy 4 profit")
print(profit4)
print("strategy 4 profit at Binance spot")
print(exchanges4['binance'].value_assets(profit4, equivalency_map))

strategy 4 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 559.3892337083817, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'PHA': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 47.38392615318298}
strategy 4 profit at Binance spot
567.063062918086


In [13]:
# for swap in flatten_swaps(all_swaps1):
#     print(swap)
# print('----------------')
# print()
# for swap in all_swaps2:
#     for ex, s in swap.items():
#         print(ex, s)